In [1]:
from rocket_functions import generate_kernels, apply_kernels, apply_kernels_jagged, run_additional(training_data, test_data, num_runs = 10, num_kernels = 10_000)
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
from sklearn.metrics import classification_report
import imblearn
import time

SyntaxError: invalid syntax (201901520.py, line 1)

In [4]:
def normalization(X_training, X_test):   
    dataref = np.concatenate((X_training, X_test), axis = 0)
    dataref_plain = dataref.reshape(dataref.shape[0]*dataref.shape[1], 1)
    X_training_plain = X_training.reshape(X_training.shape[0]*X_training.shape[1], 1)
    X_test_plain = X_test.reshape(X_test.shape[0]*X_test.shape[1], 1)
    scaler = MinMaxScaler()
    scaler.fit(dataref_plain)
    X_training_norm = scaler.transform(X_training_plain)
    X_test_norm = scaler.transform(X_test_plain)
    X_training_norm = X_training_norm.reshape(X_training.shape[0], X_training.shape[1])
    X_test_norm = X_test_norm.reshape(X_test.shape[0], X_test.shape[1])
    return X_training_norm, X_test_norm

def dataset_name_f(strrischio, overunder, normst):
    str_overunder_dataset = ''
    str_norm_dataset = ''
    if normst != '':
        str_norm_dataset = '_' + normst
    if overunder != '':
        if strrischio == '2' or strrischio == '5':
            str_overunder_dataset = '_' + overunder +'_40_60'  
        else:
            str_overunder_dataset = '_' + overunder +'_50_50'
    dataset_name = 'rischio' + strrischio + str_norm_dataset + str_overunder_dataset
    return dataset_name


def dataset_name_pad(strrischio, overunder, normst, tipo = 'train'):
    str_overunder_dataset = ''
    str_norm_dataset = ''
    if normst != '':
        str_norm_dataset = '_' + normst
    if overunder != '':
        if strrischio == '2' or strrischio == '5':
            str_overunder_dataset = '_' + overunder +'_40_60'  
        else:
            str_overunder_dataset = '_' + overunder +'_50_50'
    if tipo == 'train':
        dataset_name = 'training' + strrischio + str_norm_dataset + str_overunder_dataset + '_splitpad'
    elif tipo == 'test':
        dataset_name = 'test' + strrischio + str_norm_dataset + '_splitpad'     
    else:
        print('specifica tipo')
    return dataset_name


def read_dataset_padsplit(dataset_folder, rischio, scaling, overunder, perc_min = 40, perc_magg = 60):
    if overunder != '' and scaling != '':
        add_str_ytrain = add_str_xtrain  = '_' + scaling +'_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
    elif overunder != '' and scaling == '' :
        add_str_ytrain  = '_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
    elif overunder == '':
        add_str_ytrain = ''
    else:
        print('PROBLEMA', overunder, scaling )
        return
    with open(dataset_folder + '/pad_split/rischio' + rischio + '/' + dataset_name_pad(rischio, overunder, scaling, tipo = 'train') + '.p', 'rb') as f:
        x_train = pickle.load(f)
        X_training = x_train.astype(np.float64)
    with open(dataset_folder + '/pad_incima/rischio' + rischio + '/label_training' + rischio + add_str_ytrain + '.p', 'rb') as f:
        y_train = pickle.load(f)
        Y_training = y_train.astype(np.int32)
    with open(dataset_folder + '/pad_split/rischio' + rischio + '/' + dataset_name_pad(rischio, overunder, scaling, tipo = 'test') + '.p', 'rb') as f:
        x_test = pickle.load(f)
        X_test = x_test.astype(np.float64)
    with open(dataset_folder + '/pad_incima/rischio' + rischio + '/label_test' + rischio + '.p', 'rb') as f:
        y_test = pickle.load(f)
        Y_test = y_test.astype(np.int32)
    return X_training, Y_training, X_test, Y_test





In [5]:
def choose_kernels_perc(rischio):
    if rischio == '2' or rischio == '3':
        f = open('/Users/Administrator/Desktop/tesi/rocket-prove/kernels_2_3.pckl', 'rb')
        kernels = pickle.load(f)
        f.close()
        perc_min = 40
        perc_mag = 60    
    else:
        f = open('/Users/Administrator/Desktop/tesi/rocket-prove/kernels5.pckl', 'rb')
        kernels = pickle.load(f)
        f.close()
        perc_min = perc_mag = 50
    return kernels, perc_min, perc_mag

def run_kernels(X_training, X_test, kernels):
    inizio = time.time()
    X_training_transform = apply_kernels(X_training, kernels)
    fine = time.time()
    print('Running kernels su train:' , fine - inizio)
    inizio = time.time()
    X_test_transform = apply_kernels(X_test, kernels)
    fine = time.time()
    print('Running kernels su test:' , fine - inizio)
    X_training_transform, X_test_transform = normalization(X_training_transform, X_test_transform)
    max_iter = np.ceil(10**7 / len(X_training_transform))
    return(X_training_transform, X_test_transform, max_iter)

def overunder_str(overunder, perc_min, perc_magg):
    if overunder != '':
        overunder_p = overunder + '_' + str(perc_min) + '_' + str(perc_magg)
    else:
        overunder_p = 'no over-under'
    return(overunder_p)


def read_dataset_modified(dataset_folder, rischio, scaling, overunder, perc_min = 40, perc_magg = 60):
    if overunder != '' and scaling != '':
        add_str_ytrain = add_str_xtrain  = '_' + scaling +'_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
        add_str_xtest  = '_' + scaling
        add_str_ytest = ''
    elif overunder != '' and scaling == '' :
        add_str_ytrain = add_str_xtrain = '_' + overunder + '_' + str(perc_min) + '_' + str(perc_magg)
        add_str_ytest = add_str_xtest = ''
    elif overunder == '' and scaling != '' :
        add_str_ytrain = add_str_ytest = ''
        add_str_xtrain = add_str_xtest = '_' + scaling
    elif overunder == '' and scaling == '' :
        add_str_ytrain = add_str_ytest = add_str_xtrain = add_str_xtest = ''
    else:
        print('PROBLEMA', overunder, scaling )
        return
    with open(dataset_folder + '/training' + rischio + add_str_xtrain + '.p', 'rb') as f:
        x_train = pickle.load(f)
        X_training = x_train.astype(np.float64)
    with open(dataset_folder + '/label_training' + rischio + add_str_ytrain + '.p', 'rb') as f:
        y_train = pickle.load(f)
        Y_training = y_train.astype(np.int32)
    with open(dataset_folder + '/test' + rischio + add_str_xtest + '.p', 'rb') as f:
        x_test = pickle.load(f)
        X_test = x_test.astype(np.float64)
    with open(dataset_folder + '/label_test' + rischio + add_str_ytest + '.p', 'rb') as f:
        y_test = pickle.load(f)
        Y_test = y_test.astype(np.int32)
    return X_training, Y_training, X_test, Y_test


In [6]:
#SGD classifier with balanced accuracy
def results_SGD_balanced(rischio, perc_min, perc_magg, kernels, overunder_list, scaling_list, results_file):
    dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/'
    for scaling in scaling_list:
        for overunder in overunder_list:
            #X_training, Y_training, X_test, Y_test = read_dataset_modified(dataset_folder, rischio, scaling,
            #                                                        overunder, perc_min, perc_magg)
            X_training, Y_training, X_test, Y_test = read_dataset_padsplit(dataset_folder, rischio, scaling, 
                                                                          overunder, perc_min, perc_magg)
            X_training_transform, X_test_transform, max_iter = run_kernels(X_training, X_test, kernels)
            param_grid = {'alpha' : list(np.logspace(-3, 3, 10)), 'class_weight' : [None, 'balanced']}
            classifier = GridSearchCV(SGDClassifier(loss = 'log', max_iter = max_iter, early_stopping  = True,
                                n_iter_no_change = 12, random_state = 42), param_grid, scoring = 'balanced_accuracy')
            classifier.fit(X_training_transform, Y_training)
            train_predictions = classifier.predict(X_training_transform)
            
            overunder_p = overunder_str(overunder, perc_min, perc_magg)
            
            with open(results_file, 'a') as f:
                f.write('TRAINING' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
                        str(classifier.best_params_) + ' ' + str(classifier.scorer_)  + '\n' + str(classification_report(Y_training, train_predictions)) + '\n')
                predictions = classifier.predict(X_test_transform)
                f.write('TEST' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
                        str(classifier.best_params_) + ' ' + str(classifier.scorer_) + '\n' + str(classification_report(Y_test, predictions)) + '\n')
    return classifier

def results_SGD(rischio, perc_min, perc_magg, kernels, overunder_list, scaling_list, results_file):
    dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/'
    for scaling in scaling_list:
        for overunder in overunder_list:
            #X_training, Y_training, X_test, Y_test = read_dataset_modified(dataset_folder, rischio, scaling,
            #                                                        overunder, perc_min, perc_magg)
            
            X_training, Y_training, X_test, Y_test = read_dataset_padsplit(dataset_folder, rischio, scaling, 
                                                               overunder, perc_min, perc_magg)
            X_training_transform, X_test_transform, max_iter = run_kernels(X_training, X_test, kernels)
            
            param_grid = {'alpha' : list(np.logspace(-3, 3, 10)), 'class_weight' : [None, 'balanced']}
            classifier = GridSearchCV(SGDClassifier(loss = 'log', max_iter = max_iter, 
                    early_stopping  = True, n_iter_no_change = 12, random_state = 42), param_grid)
            inizio = time.time()
            classifier.fit(X_training_transform, Y_training)
            fine = time.time()
            print('training time: ', fine - inizio)
            inizio = time.time()
            train_predictions = classifier.predict(X_training_transform)
            fine = time.time()
            print('prediction time su training: ', fine - inizio)
            overunder_p = overunder_str(overunder, perc_min, perc_magg)
            with open(results_file, 'a') as f:
                f.write('TRAINING' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
                        str(classifier.best_params_) + '\n' + str(classification_report(Y_training, train_predictions)) + '\n')
                inizio = time.time()
                predictions = classifier.predict(X_test_transform)
                fine = time.time()
                print('prediction time su test: ', fine - inizio)
                f.write('TEST' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
                        str(classifier.best_params_) + '\n' + str(classification_report(Y_test, predictions)) + '\n')
    return classifier

In [7]:
def results_RidgeCV(rischio, perc_min, perc_magg, kernels, overunder_list, scaling_list, results_file):
    #dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/rischio' + rischio
    dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/'
    for scaling in scaling_list:
        for overunder in overunder_list:
            #X_training, Y_training, X_test, Y_test = read_dataset_modified(dataset_folder, rischio, scaling,
            #                                                        overunder, perc_min, perc_magg)
            X_training, Y_training, X_test, Y_test = read_dataset_padsplit(dataset_folder, rischio, scaling, 
                                                               overunder, perc_min, perc_magg)
            
            X_training_transform, X_test_transform, max_iter = run_kernels(X_training, X_test, kernels)
            
            if perc_min != 50:    
                class_weight_list=[None, 'balanced'] 
                scoring_list = [None, 'balanced_accuracy']
            else:
                class_weight_list = [None]
                scoring_list = [None]
            
            for scoring in scoring_list:
                for class_weight in class_weight_list:
                    classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = False,
                                                   scoring = scoring, class_weight = class_weight)
                    classifier.fit(X_training_transform, Y_training)
                    train_predictions = classifier.predict(X_training_transform)
                    overunder_p = overunder_str(overunder, perc_min, perc_magg)
                    with open(results_file, 'a') as f:
                        f.write('TRAINING' + rischio + ' ' + scaling + ' ' + overunder_p + ' alpha:' +
                                str(classifier.alpha_) + ' ' + 'class_weight: ' + str(class_weight) + 
                                ' scoring:' + str(scoring) + '\n' + str(classification_report(Y_training, train_predictions)) + '\n')
                        predictions = classifier.predict(X_test_transform)
                        f.write('TEST' + rischio + ' ' + scaling + ' ' + overunder_p + ' alpha:' +
                                str(classifier.alpha_) + ' ' + 'class_weight: ' + str(class_weight) + 
                                ' scoring:' + str(scoring) + '\n' + str(classification_report(Y_test, predictions)) + '\n')
    return

In [8]:
rischio = '5'
scaling = 'std' 
overunder = 'randover'
perc_min = '40'
perc_mag = '60'
dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/pad_incima/rischio' + rischio
X_training, Y_training, X_test, Y_test = read_dataset_modified(dataset_folder, rischio, scaling,                                                            overunder, perc_min = '40', perc_magg = '60')

In [31]:
#RIDGECV SINGOLO
X_training_transform, X_test_transform, max_iter = run_kernels(X_training, X_test, kernels)
class_weight = None
scoring = None
classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = False,
                               scoring = scoring, class_weight = class_weight)
a = time.time()
classifier.fit(X_training_transform, Y_training)
b = time.time()
print('training time', b - a)
a = time.time()
train_predictions = classifier.predict(X_training_transform)
b = time.time()
print('prediction training time', b - a)
overunder_p = overunder_str(overunder, perc_min, perc_mag)
print('TRAINING' + rischio + ' ' + scaling + ' ' + overunder_p + ' alpha:' +
        str(classifier.alpha_) + ' ' + 'class_weight: ' + str(class_weight) + 
        ' scoring:' + str(scoring) + '\n' + str(classification_report(Y_training, train_predictions)) + '\n')
a = time.time()
predictions = classifier.predict(X_test_transform)
b = time.time()
print('prediction test time', b - a)
print('TEST' + rischio + ' ' + scaling + ' ' + overunder_p + ' alpha:' +
    str(classifier.alpha_) + ' ' + 'class_weight: ' + str(class_weight) + 
    ' scoring:' + str(scoring) + '\n' + str(classification_report(Y_test, predictions)) + '\n')

training time 26.907869815826416
prediction training time 0.18318486213684082


AttributeError: 'RidgeClassifierCV' object has no attribute 'best_params_'

In [34]:
filename = '/Users/Administrator/Desktop/tesi/rocket-prove/MODELLI_SCELTI/voronoi/' + 'SDG_' + dataset_name_f(rischio, overunder, scaling) + '_modello.sav'
pickle.dump(classifier, open(filename, 'wb'))


In [33]:
X_training_transform, X_test_transform, max_iter = run_kernels(X_training, X_test, kernels)
param_grid = {'alpha' : list(np.logspace(-3, 3, 10)), 'class_weight' : [None, 'balanced']}
classifier = GridSearchCV(SGDClassifier(loss = 'log', max_iter = max_iter, 
        early_stopping  = True, n_iter_no_change = 12, random_state = 42), param_grid)
inizio = time.time()
classifier.fit(X_training_transform, Y_training)
fine = time.time()
print('training time classifier: ', fine - inizio)
inizio = time.time()
train_predictions = classifier.predict(X_training_transform)
fine = time.time()
print('prediction time classifier su training: ', fine - inizio)
overunder_p = overunder_str(overunder, perc_min, perc_mag)

print('TRAINING' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
        str(classifier.best_params_) + '\n' + str(classification_report(Y_training, train_predictions)) + '\n')
inizio = time.time()
predictions = classifier.predict(X_test_transform)
fine = time.time()
print('prediction time classifier su test: ', fine - inizio)
print('TEST' + rischio + ' ' + scaling + ' ' + overunder_p + ' ' +
        str(classifier.best_params_) + '\n' + str(classification_report(Y_test, predictions)) + '\n')

Running kernels su train: 65.61821293830872
Running kernels su test: 16.896856784820557
training time classifier:  428.3139498233795
prediction time classifier su training:  0.14063453674316406
TRAINING5 std randover_40_60 {'alpha': 0.1, 'class_weight': None}
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      2140
           1       0.92      0.93      0.93      3210

    accuracy                           0.91      5350
   macro avg       0.91      0.91      0.91      5350
weighted avg       0.91      0.91      0.91      5350


prediction time classifier su test:  0.03125333786010742
TEST5 std randover_40_60 {'alpha': 0.1, 'class_weight': None}
              precision    recall  f1-score   support

           0       0.87      0.88      0.88       477
           1       0.93      0.92      0.93       803

    accuracy                           0.91      1280
   macro avg       0.90      0.90      0.90      1280
weighted avg       0.

In [17]:
file = '/Users/Administrator/Desktop/tesi/rocket-prove/kernels_4.pckl'
filehandler = open(file,"wb")
pickle.dump(kernels,filehandler)
filehandler.close()

In [52]:
#CALCOLI NUOVI SUI DATASET PADDATI
scaling_list = ['std'] 
overunder_list = ['randover']
dataset_folder = '/Users/Administrator/Desktop/tesi/datasets_tesi/'
results_file = '/Users/Administrator/Desktop/tesi/rocket-prove/rocket_trials_results/pad_split/SGD_rischio' + rischio + '.txt'
classifier = results_SGD(rischio, perc_min, perc_mag, kernels, overunder_list, scaling_list, results_file)
filename = '/Users/Administrator/Desktop/tesi/rocket-prove/modelli_salvati/pad_split/' + 'SGD_' + dataset_name_f(rischio, overunder, scaling) + '_modello.sav'
pickle.dump(classifier, open(filename, 'wb'))

/Users/Administrator/Desktop/tesi/datasets_tesi/


In [53]:
classifier = results_SGD_balanced(rischio, perc_min, perc_mag, kernels, overunder_list, scaling_list, results_file)

/Users/Administrator/Desktop/tesi/datasets_tesi/


In [54]:
results_file = '/Users/Administrator/Desktop/tesi/rocket-prove/rocket_trials_results/pad_split/ridgeCV_rischio' + rischio + '.txt'
results_RidgeCV(rischio, perc_min, perc_mag, kernels, overunder_list, scaling_list, results_file)

/Users/Administrator/Desktop/tesi/datasets_tesi/


In [7]:
rischio_list = ['2'] 
for rischio in rischio_list:
    kernels, perc_min, perc_mag = choose_kernels_perc(rischio)
    output = '/Users/Administrator/Desktop/tesi/rocket-prove/rocket_trials_results/SDG_rischio' + rischio + '.txt'
    results_SGD(rischio, perc_min, perc_mag, kernels, output)
    if perc_min!=50:
        results_SGD_balanced(rischio, perc_min, perc_mag, kernels, output)
    

In [33]:
f = open('kernels.pckl', 'rb')
kernels = pickle.load(f)
f.close()

f = open('kernels2.pckl', 'rb')
kernels2 = pickle.load(f)
f.close()

rischio_list = ['2', '3', '4' , '5'] 
for rischio in rischio_list:
    if rischio == '2' or rischio == '5':
        perc_min = 40
        perc_mag = 60
        kernels = kernels
    else:
        perc_min = perc_mag = 50
        kernels = kernels2
    output = '/Users/Administrator/Desktop/tesi/rocket-prove/ridgeCV_rischio' + rischio + '.txt'
    results_RidgeCV(rischio, perc_min, perc_mag, kernels, output)

In [7]:
f = open('kernels.pckl', 'rb')
kernels = pickle.load(f)
f.close()

f = open('kernels2.pckl', 'rb')
kernels2 = pickle.load(f)
f.close()

rischio_list = ['2', '3', '4' , '5'] 
for rischio in rischio_list:
    if rischio == '2' or rischio == '5':
        perc_min = 40
        perc_mag = 60
        kernels = kernels
    else:
        perc_min = perc_mag = 50
        kernels = kernels2
    output = '/Users/Administrator/Desktop/tesi/rocket-prove/ridgeClassPG_rischio' + rischio + '.txt'
    results_RidgeClassPG(rischio, perc_min, perc_mag, kernels, output)

"f = open('kernels.pckl', 'rb')\nkernels = pickle.load(f)\nf.close()\n\nf = open('kernels2.pckl', 'rb')\nkernels2 = pickle.load(f)\nf.close()\n\nrischio_list = ['2', '3', '4' , '5'] \nfor rischio in rischio_list:\n    if rischio == '2' or rischio == '5':\n        perc_min = 40\n        perc_mag = 60\n        kernels = kernels\n    else:\n        perc_min = perc_mag = 50\n        kernels = kernels2\n    output = '/Users/Administrator/Desktop/tesi/rocket-prove/ridgeClassPG_rischio' + rischio + '.txt'\n    results_RidgeClassPG(rischio, perc_min, perc_mag, kernels, output)"

In [10]:
f = open('kernels_5k.pckl', 'rb')
kernels = pickle.load(f)
f.close()
perc_min = 40
perc_mag = 60
rischio_list = ['2'] 
output = '/Users/Administrator/Desktop/tesi/rocket-prove/rocket_trials_results/SDG_5k_rischio' + rischio + '.txt'
results_SGD(rischio, perc_min, perc_mag, kernels, output)
if perc_min!=50:
    results_SGD_balanced(rischio, perc_min, perc_mag, kernels, output)